## MLflow's Model Registry

In [3]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [5]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


In [10]:
client.get_experiment

<bound method MlflowClient.get_experiment of <mlflow.tracking.client.MlflowClient object at 0x7c093e0891f0>>

In [8]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [11]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [12]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 128d98200b994438ab27fe1a635e7be2, rmse: 6.3051
run id: 556113198a5743fb824c0eeeb8fabf96, rmse: 6.3084
run id: 54fb0b6cc64d4d2989b05cb3428476bc, rmse: 6.3087
run id: abd7f98fed35492b81daece1024561f0, rmse: 6.3110
run id: 2169809b43a6445595911f82ea4c50e3, rmse: 6.3120


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [13]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [14]:
run_id = "128d98200b994438ab27fe1a635e7be2"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Successfully registered model 'nyc-taxi-regressor'.
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716835252876, current_stage='None', description=None, last_updated_timestamp=1716835252876, name='nyc-taxi-regressor', run_id='128d98200b994438ab27fe1a635e7be2', run_link=None, source='/workspaces/MLOps-zoomcamp-2024/02-experiment-tracking/mlruns/1/128d98200b994438ab27fe1a635e7be2/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [15]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_23969/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [16]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_23969/1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716835252876, current_stage='Staging', description=None, last_updated_timestamp=1716835469451, name='nyc-taxi-regressor', run_id='128d98200b994438ab27fe1a635e7be2', run_link=None, source='/workspaces/MLOps-zoomcamp-2024/02-experiment-tracking/mlruns/1/128d98200b994438ab27fe1a635e7be2/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [17]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716835252876, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-05-27', last_updated_timestamp=1716835552785, name='nyc-taxi-regressor', run_id='128d98200b994438ab27fe1a635e7be2', run_link=None, source='/workspaces/MLOps-zoomcamp-2024/02-experiment-tracking/mlruns/1/128d98200b994438ab27fe1a635e7be2/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [20]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_23969/2437174745.py", line 1, in <module>
    from sklearn.metrics import mean_squared_error
  File "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/__init__.py", line 96, in <module>
    from .base import clone
  File "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/base.py", line 19, in <module>
    from .utils._estimator_html_repr import _HTMLDocumentationLinkMixin, estimator_html_repr
  File "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/utils/__init__.py", line 13, in <module>
    from ._chunking import gen_batches, gen_even_slices
  File "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/u

In [21]:
df = read_dataframe("data/green_tripdata_2021-03.csv")

NameError: name 'read_dataframe' is not defined

In [37]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/cristian.martinez/Repositories/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [38]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [39]:
X_test = preprocess(df, dv)

In [40]:
target = "duration"
y_test = df[target].values

In [41]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 139 ms, sys: 44.6 ms, total: 183 ms
Wall time: 447 ms


{'rmse': 6.659623830022514}

In [42]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 6.94 s, sys: 216 ms, total: 7.16 s
Wall time: 7.28 s


{'rmse': 6.881555517147188}

In [22]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_23969/1316468422.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716835252876, current_stage='Production', description='The model version 1 was transitioned to Staging on 2024-05-27', last_updated_timestamp=1716835963505, name='nyc-taxi-regressor', run_id='128d98200b994438ab27fe1a635e7be2', run_link=None, source='/workspaces/MLOps-zoomcamp-2024/02-experiment-tracking/mlruns/1/128d98200b994438ab27fe1a635e7be2/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>